In [2]:
source('libraries.R')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tidyr’


The following object is masked from ‘package:magrittr’:

    extract


Registered S3 methods overwritten by 'ggtern':
  method           from   
  grid.draw.ggplot ggplot2
  plot.ggplot      ggplot2
  print.ggplot     ggplot2

--
Remember to cite, run citation(package = 'ggtern') for further info.
--


Attaching package: ‘ggtern’


The following objects are masked from ‘package:ggplot2’:

    aes, annotate, ggplot, ggplot_build, ggplot_gtable, ggplotGrob,
    ggsave, layer_data, theme_bw, theme_classic, theme_dark,
    theme_gray, theme_light, theme_linedraw, theme_minimal, theme_void


Loading required package: viridisLite


Attaching package: ‘purrr’


The following obje

In [112]:
cdr3_hla_matrix <- fread('../cdr3_hla_pairs_9PCs/DRB1:13:12:P113_matrix.tsv')

In [111]:
head(cdr3_hla_matrix)

patient_id,allele_R,allele_Y,allele_S,allele_G,allele_H,allele_F,pair,L,F,⋯,N,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9
<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
H0,2,0,0,0,0,0,DRB1:13:12:P113,2.26662553,1.4837426,⋯,0.3799881,-0.028780271,0.040238904,-0.0071729478,0.033242921,-0.010985428,0.001170305,-0.0053977252,0.0032817939,-0.0097599940
H1,1,1,0,0,0,0,DRB1:13:12:P113,0.95465882,-0.2794275,⋯,0.4728554,-0.020513447,0.013880925,-0.0007512883,-0.009227345,0.029113552,0.007404630,-0.0370105065,-0.0074489374,0.0137528756
H10,0,0,2,0,0,0,DRB1:13:12:P113,0.25644576,-1.0793381,⋯,1.4273260,-0.021797512,-0.014022404,0.0125275207,0.008571864,-0.009477230,-0.041021895,-0.0001910232,-0.0004914357,-0.0071156440
H100,0,0,2,0,0,0,DRB1:13:12:P113,0.03195778,1.0009366,⋯,-1.3108562,-0.004942368,-0.002447521,-0.0306362403,0.003272312,-0.020144411,-0.033990572,-0.0070075715,0.0020702732,-0.0053175130
H101,1,0,1,0,0,0,DRB1:13:12:P113,0.20421001,-0.1670564,⋯,-0.8888392,-0.027056046,-0.009474266,0.0148949435,-0.021221340,-0.004339928,0.015587497,-0.0359386456,-0.0061768543,-0.0007301164
H102,0,0,1,1,0,0,DRB1:13:12:P113,0.57922856,-0.8869050,⋯,0.1683790,-0.026334089,-0.015815421,0.0198324257,0.009153323,-0.006428599,0.017137351,-0.0401447019,-0.0076326677,0.0015133883


### without setting alleles as factors

In [44]:
anova(mod1, mod0)

,Res.Df,Df,Gen.var.,Pillai,approx F,num Df,den Df,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1883,NA,0.6441223,NA,NA,NA,NA,NA
2,1888,5,0.6702969,0.7105947,16.29598,95,9345,5.297502e-239


### removing the reference

In [113]:
#cdr3_hla_matrix <- cdr3_hla_matrix %>%
  #mutate(across(contains("allele"), as.factor))
#cdr3_hla_matrix$S <- as.factor(cdr3_hla_matrix$S)
#cdr3_hla_matrix$S <- relevel(cdr3_hla_matrix$S, ref = 2)
amino_acids <-c('A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y') 

hla_alleles <- colnames(cdr3_hla_matrix %>% dplyr::select(contains('allele')))
aa_in_matrix <- paste0(colnames(cdr3_hla_matrix)[colnames(cdr3_hla_matrix) %in% amino_acids], collapse = ',')
Y <- paste0('cbind(',aa_in_matrix,') ~')
X <- paste0(hla_alleles[-3], collapse = '+')
PCs <- paste0('PC', seq(1,9), collapse = '+')

formula_full <- as.formula(str_c(paste0(Y,X),PCs, sep   = '+'))
formula_null <- as.formula(str_c(Y,PCs))
    

mod1 <- lm(formula_full, data = cdr3_hla_matrix)
mod0 <- lm(formula_null, data = cdr3_hla_matrix)
anova(mod1, mod0)

,Res.Df,Df,Gen.var.,Pillai,approx F,num Df,den Df,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1883,NA,0.6441223,NA,NA,NA,NA,NA
2,1888,5,0.6702969,0.7105947,16.29598,95,9345,5.297502e-239


In [115]:
DF_full <- df.residual(mod1)
DF_null <- df.residual(mod0)
DF_full
DF_null
#response variable matrix
Y <- cdr3_hla_matrix %>% dplyr::select(any_of(amino_acids)) %>% mutate(across(everything(), as.numeric))
Y <- as.matrix(Y)
dim(Y)
head(Y)
mod1_fitted_ordered <- mod1$fitted.values[, colnames(Y)]

# Step 3: Subtract the values element-wise
Res_full <- Y - mod1_fitted_ordered

# Step 4: Check the dimensions of the result
dim(Res_full)
mod0_fitted_ordered <- mod0$fitted.values[, colnames(Y)]

# Step 3: Subtract the values element-wise
Res_null <- Y - mod0_fitted_ordered

# Step 4: Check the dimensions of the result
dim(Res_null)

Emat <- crossprod(Res_full)
Hmat <- crossprod(Res_null) - crossprod(Res_full)
dim(Emat)
dim(Hmat)

Pillai <- sum(diag( Hmat %*% solve( Hmat + Emat ) ))
Pillai

p=19 #DF of Y matrix (N of amino acids)
q=6 #DF of X (dose1-dose5)
s=min(p,q)
v=DF_full
m=(abs(p-q)-1)/2
n=(v-p-1)/2
appF <- (2*n + s + 1)/(2*m + s + 1) * ( Pillai / (s - Pillai) )
appF #the identical value as R function (see above)

numDF <- s*(2*n + s + 1)
numDF #the identical value as R function (see above)

dnomDF <- s*(2*m + s + 1)
dnomDF #the identical value as R function (see above)

#obtained the same P value!!
pf(appF, dnomDF,numDF, lower.tail = FALSE) #the identical value as R function (see above)

[1] 1883

[1] 1888

[1] 1898   19

A,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
1.0793381,-1.9136009,-2.1746466,1.4837426,-1.3327244,-0.2470226,1.2044702,0.8451575,2.26662553,1.9631024,0.3799881,1.29863658,-0.6909544,0.07898476,-1.0472046,1.2044702,2.0639564,0.2685949,-0.89466210
1.0700443,-1.1779570,-0.8677414,-0.2794275,-0.1512067,-1.2346481,0.4963223,0.2998285,0.95465882,-0.3383364,0.4728554,0.10780305,0.7415225,-0.20953956,0.8266619,0.1564855,0.5066749,0.9923645,-1.19642780
0.3799881,-0.2002167,0.4366973,-1.0793381,0.6388926,-0.6485190,0.5260473,1.8450077,0.25644576,-0.1863859,1.4273260,-0.70928479,-0.2148751,-0.63410152,0.2042100,-1.2430762,0.4670296,0.0750606,-1.38526012
1.1102079,0.0515378,0.6909544,1.0009366,-1.5807124,-1.3886725,0.1657340,2.0980571,0.03195778,0.3887080,-1.3108562,-1.08637007,-0.3940442,-0.07113760,-0.2916536,0.5155919,1.1991000,1.2402571,1.77146848
-0.6420948,-0.1856012,-0.5962795,-0.1670564,-0.4295352,0.1988863,1.7590796,0.8248279,0.20421001,0.9896166,-0.8888392,0.14593206,0.3176105,-0.72276691,0.5730700,1.2866078,0.8451575,0.2699473,0.03065296
1.5410318,-0.6469104,-0.4919018,-0.8869050,1.5672102,-0.4904304,-0.3827933,-1.4493407,0.57922856,-0.2915412,0.1683790,-0.04370384,0.3437832,-0.99878672,-1.5583651,0.9222473,-1.2487444,0.7025919,0.28349814


[1] 1898   19

...

[1] 0.7105947

[1] 13.22218

[1] 11220

[1] 114

[1] 1.75751e-224

In [25]:
df_long <- cdr3_hla_matrix %>%
  pivot_longer(cols = starts_with("allele_"), 
               names_to = "allele_type", 
               values_to = "allele_value")
sum(df_long$allele_value)

[1] 3796

In [21]:
# Step 2: Group by patient_id and concatenate allele values
df_hla <- df_long %>%
  group_by(patient_id) %>%
  mutate(HLA = paste(rep(allele_type, allele_value), collapse = "")) %>%
  dplyr::select(-c(allele_type, allele_value)) %>%
  distinct()

df_hla

patient_id,pair,L,F,G,S,Y,A,P,R,⋯,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,HLA
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
H0,DRB1:13:12:P113,2.26662553,1.48374263,-1.33272438,-1.04720465,-0.894662105,1.07933808,1.29863658,0.07898476,⋯,-0.028780271,0.040238904,-0.0071729478,0.033242921,-0.010985428,0.001170305,-0.0053977252,0.0032817939,-0.0097599940,allele_Rallele_R
H1,DRB1:13:12:P113,0.95465882,-0.27942752,-0.15120666,0.82666190,-1.196427804,1.07004428,0.10780305,-0.20953956,⋯,-0.020513447,0.013880925,-0.0007512883,-0.009227345,0.029113552,0.007404630,-0.0370105065,-0.0074489374,0.0137528756,allele_Rallele_Y
H10,DRB1:13:12:P113,0.25644576,-1.07933808,0.63889257,0.20421001,-1.385260115,0.37998815,-0.70928479,-0.63410152,⋯,-0.021797512,-0.014022404,0.0125275207,0.008571864,-0.009477230,-0.041021895,-0.0001910232,-0.0004914357,-0.0071156440,allele_Sallele_S
H100,DRB1:13:12:P113,0.03195778,1.00093664,-1.58071241,-0.29165357,1.771468476,1.11020785,-1.08637007,-0.07113760,⋯,-0.004942368,-0.002447521,-0.0306362403,0.003272312,-0.020144411,-0.033990572,-0.0070075715,0.0020702732,-0.0053175130,allele_Sallele_S
H101,DRB1:13:12:P113,0.20421001,-0.16705636,-0.42953515,0.57307001,0.030652960,-0.64209477,0.14593206,-0.72276691,⋯,-0.027056046,-0.009474266,0.0148949435,-0.021221340,-0.004339928,0.015587497,-0.0359386456,-0.0061768543,-0.0007301164,allele_Rallele_S
H102,DRB1:13:12:P113,0.57922856,-0.88690496,1.56721024,-1.55836511,0.283498143,1.54103177,-0.04370384,-0.99878672,⋯,-0.026334089,-0.015815421,0.0198324257,0.009153323,-0.006428599,0.017137351,-0.0401447019,-0.0076326677,0.0015133883,allele_Sallele_G
H103,DRB1:13:12:P113,-0.72276691,-0.54561911,0.94646279,-0.34239992,-0.942388532,-0.55776755,0.84143540,-0.30802341,⋯,-0.023841642,0.023411343,0.0111188053,0.025952859,0.029771182,-0.003477018,-0.0042996923,-0.0179570482,-0.0004667786,allele_Yallele_H
H104,DRB1:13:12:P113,0.44676212,1.33590084,-0.80123281,-0.37298833,0.475774313,0.09469417,-1.81711993,0.84143540,⋯,-0.031151461,0.041885793,-0.0095821616,0.031837422,-0.016874565,-0.001633081,-0.0033757865,-0.0140596194,0.0207020647,allele_Sallele_S
H105,DRB1:13:12:P113,-0.81025442,-1.54103177,-0.58540916,-0.83957871,0.360434310,-0.30802341,-0.18162211,0.39404423,⋯,-0.018733767,0.017177212,-0.0244145564,0.012371418,-0.025487335,0.008786448,-0.0312218685,-0.0214745960,0.0501780332,allele_Rallele_S


In [30]:
df_hla$HLA <- factor(df_hla$HLA)
df_hla$HLA <- relevel(df_hla$HLA, ref = 'allele_Sallele_S')

In [62]:
amino_acids <-c('A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y') 
aa_in_matrix <- paste0(colnames(df_hla)[colnames(df_hla) %in% amino_acids], collapse = ',')
Y <- paste0('cbind(',aa_in_matrix,') ~')
X <- 'HLA'
PCs <- paste0('PC', seq(1,9), collapse = '+')

In [63]:
formula_full <- as.formula(str_c(paste0(Y,X),PCs, sep   = '+'))
formula_null <- as.formula(str_c(Y,PCs))

In [64]:
formula_full

cbind(L, F, G, S, Y, A, P, R, Q, I, D, W, T, E, H, K, V, M, N) ~ 
    HLA + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9

In [65]:
#full model (exclude S_allele, the most frequent allele, as the reference allele)
mod1 <- lm(formula_full, 
    data = df_hla)

#null model
mod0 <- lm(formula_null,  
    data = df_hla)

In [66]:
anova(mod1, mod0)

,Res.Df,Df,Gen.var.,Pillai,approx F,num Df,den Df,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1868,NA,0.6404496,NA,NA,NA,NA,NA
2,1888,20,0.6702969,0.9552772,4.944542,380,35492,7.900376e-190
